## Deliverable 2. Create a Customer Travel Destinations Map.

In [53]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

SyntaxError: invalid syntax (<ipython-input-53-d3ddbde52b11>, line 8)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/Capit/Desktop/UC Berkeley Data Boot Camp/Analysis Projects/Module 6/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.87,113.63,62.60,77,0,14.99,clear sky
1,1,Yar-Sale,RU,66.83,70.83,50.09,93,26,12.21,scattered clouds
2,2,Mataura,NZ,-46.19,168.86,44.01,94,12,1.99,few clouds
3,3,Looc,PH,12.26,121.99,81.88,79,100,6.33,overcast clouds
4,4,Port Elizabeth,ZA,-33.92,25.57,60.80,77,97,5.82,overcast clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Looc,PH,12.26,121.99,81.88,79,100,6.33,overcast clouds
7,7,Neiafu,TO,-18.65,-173.98,78.80,83,20,10.29,few clouds
8,8,Kapaa,US,22.08,-159.32,86.00,70,40,10.29,scattered clouds
9,9,Tual,ID,-5.67,132.75,79.03,78,63,17.31,broken clouds
12,12,Hasaki,JP,35.73,140.83,75.20,100,75,6.93,mist
13,13,Sola,VU,-13.88,167.55,80.98,83,72,9.95,light rain
18,18,Kalianget,ID,-7.05,113.93,78.49,81,7,12.80,clear sky
19,19,San Cristobal,VE,7.77,-72.22,87.80,45,40,19.46,scattered clouds
20,20,Puerto Escondido,MX,15.85,-97.07,80.60,74,90,5.82,moderate rain
23,23,Thinadhoo,MV,0.53,72.93,83.21,74,73,10.22,broken clouds


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                254
City                   254
Country                254
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# I didn't find any empty rows

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Looc,PH,81.88,overcast clouds,12.26,121.99,
7,Neiafu,TO,78.80,few clouds,-18.65,-173.98,
8,Kapaa,US,86.00,scattered clouds,22.08,-159.32,
9,Tual,ID,79.03,broken clouds,-5.67,132.75,
12,Hasaki,JP,75.20,mist,35.73,140.83,
13,Sola,VU,80.98,light rain,-13.88,167.55,
18,Kalianget,ID,78.49,clear sky,-7.05,113.93,
19,San Cristobal,VE,87.80,scattered clouds,7.77,-72.22,
20,Puerto Escondido,MX,80.60,moderate rain,15.85,-97.07,
23,Thinadhoo,MV,83.21,broken clouds,0.53,72.93,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
hotel_df["Hotel Name"].isnull().sum()

0

In [43]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('',np.nan, inplace=True)
print (hotel_df)

                    City Country  Max Temp Current Description    Lat     Lng  \
3                   Looc      PH     81.88     overcast clouds  12.26  121.99   
7                 Neiafu      TO     78.80          few clouds -18.65 -173.98   
8                  Kapaa      US     86.00    scattered clouds  22.08 -159.32   
9                   Tual      ID     79.03       broken clouds  -5.67  132.75   
12                Hasaki      JP     75.20                mist  35.73  140.83   
..                   ...     ...       ...                 ...    ...     ...   
677              Aquiraz      BR     80.60     overcast clouds  -3.90  -38.39   
679       Puerto Morazan      NI     80.60       moderate rain  12.85  -87.17   
685              Bintulu      MY     75.20          light rain   3.17  113.03   
687              Anuling      PH     77.50          few clouds   6.04  121.01   
688  Santiago Del Estero      AR     76.87     overcast clouds -27.80  -64.26   

                           

In [44]:
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
print (hotel_df)

                    City Country  Max Temp Current Description    Lat     Lng  \
3                   Looc      PH     81.88     overcast clouds  12.26  121.99   
7                 Neiafu      TO     78.80          few clouds -18.65 -173.98   
8                  Kapaa      US     86.00    scattered clouds  22.08 -159.32   
9                   Tual      ID     79.03       broken clouds  -5.67  132.75   
12                Hasaki      JP     75.20                mist  35.73  140.83   
..                   ...     ...       ...                 ...    ...     ...   
677              Aquiraz      BR     80.60     overcast clouds  -3.90  -38.39   
679       Puerto Morazan      NI     80.60       moderate rain  12.85  -87.17   
685              Bintulu      MY     75.20          light rain   3.17  113.03   
687              Anuling      PH     77.50          few clouds   6.04  121.01   
688  Santiago Del Estero      AR     76.87     overcast clouds -27.80  -64.26   

                           

In [50]:
hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Looc,PH,81.88,overcast clouds,12.26,121.99,Cummings Highlands
7,Neiafu,TO,78.80,few clouds,-18.65,-173.98,Mystic Sands
8,Kapaa,US,86.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,Tual,ID,79.03,broken clouds,-5.67,132.75,Grand Vilia Hotel
12,Hasaki,JP,75.20,mist,35.73,140.83,Inubosaki Kanko Hotel
13,Sola,VU,80.98,light rain,-13.88,167.55,Leumerus Bungalows
18,Kalianget,ID,78.49,clear sky,-7.05,113.93,Pesanggrahan Garam
19,San Cristobal,VE,87.80,scattered clouds,7.77,-72.22,Pirineos
20,Puerto Escondido,MX,80.60,moderate rain,15.85,-97.07,Aldea del Bazar Hotel and Spa
27,Palmer,US,90.00,broken clouds,42.16,-72.33,Wedgewood Motel


In [52]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))